# **Fine-Tuning and Inference with Base YOLOv5 Model on Our Gun Dataset**

In [ ]:
from google.colab import drive
import cv2
import os
import json

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Dataset Preprocessing: Renaming and Organizing into Train/Validation Splits**

In [ ]:
from sklearn.model_selection import train_test_split
import os
import shutil
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

def ensure_dir(directory):
    """Ensure directory exists."""
    os.makedirs(directory, exist_ok=True)
    print(f"Ensured directory: {directory}")

# Function to copy files to destination folders
def copy_files(files, dest_folder):
    for file in files:
        basename = os.path.basename(file)
        dest_file = os.path.join(dest_folder, basename)
        shutil.copy(file, dest_file)
        print(f"Copied {file} to {dest_file}")

# Define paths
base_dir = '/content/drive/My Drive/DL/Renamed_Frames'
dest_base_dir = '/content/drive/My Drive/DL/Model'

# Separate image and annotation files by category
category_files = {'Handgun': [], 'Machine_Gun': []}

# Walk through the directory structure and separate files by category
for dirpath, dirnames, filenames in os.walk(base_dir):
    for filename in filenames:
        if filename.endswith('.jpg'):
            annotation_file = os.path.splitext(filename)[0] + '.txt'
            annotation_path = os.path.join(dirpath, annotation_file)
            if os.path.isfile(annotation_path):
                if 'Handgun' in dirpath:
                    category_files['Handgun'].append((os.path.join(dirpath, filename), annotation_path))
                elif 'Machine_Gun' in dirpath:
                    category_files['Machine_Gun'].append((os.path.join(dirpath, filename), annotation_path))

# Stratified split for each category
for category, files in category_files.items():
    images, annotations = zip(*files)
    images_train, images_val, annotations_train, annotations_val = train_test_split(
        images, annotations, test_size=0.3, random_state=42
    )

    # Define destination folders for each category
    train_images_dest = os.path.join(dest_base_dir, 'train', 'images', category)
    train_labels_dest = os.path.join(dest_base_dir, 'train', 'labels', category)
    val_images_dest = os.path.join(dest_base_dir, 'val', 'images', category)
    val_labels_dest = os.path.join(dest_base_dir, 'val', 'labels', category)

    # Ensure directories
    ensure_dir(train_images_dest)
    ensure_dir(train_labels_dest)
    ensure_dir(val_images_dest)
    ensure_dir(val_labels_dest)

    # Copy the files to their respective directories
    copy_files(images_train, train_images_dest)
    copy_files(annotations_train, train_labels_dest)
    copy_files(images_val, val_images_dest)
    copy_files(annotations_val, val_labels_dest)

print("Data split and organized successfully.")


Mounted at /content/drive
Ensured directory: /content/drive/My Drive/DL/Model/train/images/Handgun
Ensured directory: /content/drive/My Drive/DL/Model/train/labels/Handgun
Ensured directory: /content/drive/My Drive/DL/Model/val/images/Handgun
Ensured directory: /content/drive/My Drive/DL/Model/val/labels/Handgun
Copied /content/drive/My Drive/DL/Renamed_Frames/Handgun/PAH1_C1_P2_V1_HB_3/Handgun_674.jpg to /content/drive/My Drive/DL/Model/train/images/Handgun/Handgun_674.jpg
Copied /content/drive/My Drive/DL/Renamed_Frames/Handgun/PAH1_C1_P4_V1_HB_2/Handgun_2177.jpg to /content/drive/My Drive/DL/Model/train/images/Handgun/Handgun_2177.jpg
Copied /content/drive/My Drive/DL/Renamed_Frames/Handgun/PAH1_C1_P1_V1_HB_4/Handgun_436.jpg to /content/drive/My Drive/DL/Model/train/images/Handgun/Handgun_436.jpg
Copied /content/drive/My Drive/DL/Renamed_Frames/Handgun/PAH1_C2_P3_V2_HB_3/Handgun_961.jpg to /content/drive/My Drive/DL/Model/train/images/Handgun/Handgun_961.jpg
Copied /content/drive/My

In [ ]:
# Mount Google Drive to access dataset and save models

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change directory to the model's location

cd /content/drive/MyDrive/DL/Model

/content/drive/MyDrive/DL/Model


In [ ]:
# Set base directory for the gun dataset
base_dir='/content/drive/MyDrive/DL/Gun_Dataset'

In [ ]:
frames_base_dir = '/content/drive/MyDrive/DL/Renamed_Frames'

In [ ]:
import os
import json
import cv2

In [ ]:
# Change directory to the folder with renamed frames

cd /content/drive/MyDrive/DL/Renamed_Frames

/content/drive/MyDrive/DL/Renamed_Frames


In [ ]:
# List contents of the current directory
ls

Handgun/  Machine_Gun/


In [ ]:
# Change back to the model's directory
cd /content/drive/MyDrive/DL/Model/

/content/drive/MyDrive/DL/Model


In [ ]:
ls

dataset.yaml  train/  val/


In [ ]:
# Clone the YOLOv5 repository from GitHub

!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 16582 (delta 30), reused 39 (delta 19), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.06 MiB | 22.45 MiB/s, done.
Resolving deltas: 100% (11389/11389), done.


In [ ]:
# Change directory to the yolov5 folder

cd yolov5

/content/drive/MyDrive/DL/Model/yolov5


In [ ]:
# Install required dependencies

!pip install -r requirements.txt

In [ ]:
import torch
from matplotlib import pyplot as plt
import cv2
import numpy as np

In [ ]:
# Load the YOLOv5 model pre-trained

model=torch.hub.load('ultralytics/yolov5','yolov5s')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-5-6 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

100%|██████████| 14.1M/14.1M [00:00<00:00, 160MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoS

In [ ]:
# Change directory to the yolov5 folder

cd /content/drive/MyDrive/DL/Model/yolov5

/content/drive/MyDrive/DL/Model/yolov5


In [ ]:
ls

benchmarks.py    data/       LICENSE         README.zh-CN.md   tutorial.ipynb
CITATION.cff     detect.py   models/         requirements.txt  utils/
classify/        export.py   pyproject.toml  segment/          val.py
CONTRIBUTING.md  hubconf.py  README.md       train.py          yolov5s.pt


In [ ]:
# Fine-tune the YOLOv5 model on our gun dataset

!python train.py --img 480 --batch 16 --epochs 10 --data /content/drive/MyDrive/DL/Model/dataset.yaml --weights yolov5s.pt --cache

2024-05-06 05:05:12.330987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 05:05:12.331039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 05:05:12.332516: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/DL/Model/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=10, batch_size=16, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_sc

In [ ]:
# Perform inference using the fine-tuned YOLOv5 model on a handgun video

!python "/content/drive/MyDrive/DL/Model/yolov5/detect.py" --source "/content/drive/My Drive/DL/Gun_Dataset/Handgun/PAH1_C1_P1_V1_HB_3/video.mp4" --weights "/content/drive/MyDrive/DL/Model/yolov5/runs/train/exp/weights/best.pt"

detect: weights=['/content/drive/MyDrive/DL/Model/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/My Drive/DL/Gun_Dataset/Handgun/PAH1_C1_P1_V1_HB_3/video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-307-g920c721e Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/150) /content/drive/My Drive/DL/Gun_Dataset/Handgun/PAH1_C1_P1_V1_HB_3/video.mp4: 480x640 (no detections), 85.5ms
video 1/1 (2/150) /content/drive/My Drive/DL/Gun_Dataset/Handgun/PAH1_C1_P1_V1_HB_3/video.mp4: 

In [ ]:
# Perform inference using the fine-tuned YOLOv5 model on a machine gun video


!python "/content/drive/MyDrive/DL/Model/yolov5/detect.py" --source "/content/drive/My Drive/DL/Gun_Dataset/Machine_Gun/PAW2_C1_P1_V1_HB_3/video.mp4" --weights "/content/drive/MyDrive/DL/Model/yolov5/runs/train/exp/weights/best.pt"

detect: weights=['/content/drive/MyDrive/DL/Model/yolov5/runs/train/exp/weights/best.pt'], source=/content/drive/My Drive/DL/Gun_Dataset/Machine_Gun/PAW2_C1_P1_V1_HB_3/video.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-307-g920c721e Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/150) /content/drive/My Drive/DL/Gun_Dataset/Machine_Gun/PAW2_C1_P1_V1_HB_3/video.mp4: 480x640 (no detections), 81.8ms
video 1/1 (2/150) /content/drive/My Drive/DL/Gun_Dataset/Machine_Gun/PAW2_C1_P1_V1_HB_3